# Data loading 

In [1]:
import pandas as pd
import numpy as np 
import scipy.sparse as sps
import os
import sys
import matplotlib.pyplot as plt
sys.path.append('..')


In [2]:
test_users = pd.read_csv('data_target_users_test.csv')

#array_test_users = test_users.copy().to_numpy().squeeze()
#array_test_users

array([   0,    1,    2, ..., 7944, 7945, 7946])

In [16]:
#data which contains users (row), items(col) and implicit interaction (data)
dataset = pd.read_csv('data_train.csv')
#dataset = dataset[dataset['row'].isin(array_test_users)]
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


# Data pre-processing
Pre-processing of data to check for missing users, items ecc...

In [4]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all

<7947x25975 sparse matrix of type '<class 'numpy.float64'>'
	with 113265 stored elements in Compressed Sparse Row format>

In [5]:
ICM_df = pd.read_csv('data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [6]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [7]:
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [8]:
from GraphBased.P3alphaRecommender import P3alphaRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender


recommender_itemKNNCBF = ItemKNNCBFRecommender(URM_all, ICM_all)
recommender_itemKNNCBF.fit(topK=600, shrink=200, feature_weighting = 'TF-IDF')


ItemKNNCBFRecommender: URM Detected 3 (0.04 %) cold users.
ItemKNNCBFRecommender: URM Detected 1079 (4.15 %) cold items.
Similarity column 25975 ( 100 % ), 3539.63 column/sec, elapsed time 0.12 min


In [9]:

user_id = test_users['user_id']
#user_id = np.arange(start=0, stop=7947, step=1)
print(user_id)
recommendations, scores = recommender_itemKNNCBF.recommend(user_id, return_scores = True)


In [10]:

scores[~np.isfinite(scores)] = 0
scores = sps.csr_matrix(scores)
recommender_user_cf = UserKNNCFRecommender(scores)
recommender_user_cf.fit(shrink=100, topK=150, feature_weighting = "TF-IDF")


UserKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
Similarity column 1589 ( 20 % ), 52.37 column/sec, elapsed time 0.51 min
Similarity column 4770 ( 60 % ), 52.85 column/sec, elapsed time 1.50 min
Similarity column 7944 ( 100 % ), 53.73 column/sec, elapsed time 2.46 min


In [11]:
print(scores.shape)

(7944, 25975)


In [12]:
print(user_id.shape)

(7944,)


In [17]:


recommendations = recommender_user_cf.recommend(user_id.reset_index(),cutoff = 10)

IndexError: index (7946) out of range

In [ ]:
for index in range(len(recommendations)):
    #print(element)
    recommendations[index]=np.array(recommendations[index])
    #print(type(element))
print(len(recommendations))

In [ ]:
test_users['item_list']= recommendations
#test_users['item_list'] =  test_users['item_list'].apply(lambda x: x.replace('[','').replace(']','')) 
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])

#convert the string columns to int
#test_users['item_list'] = test_users['item_list'].astype(int)
test_users
test_users.to_csv('submission.csv', index=False)
